# Data

Let's have a look at the data first

In [ ]:
import os
from pathlib import Path

input_data_path = Path(os.environ.get('INPUT_DATA_PATH', '.'))
output_data_path = Path(os.environ.get('OUTPUT_DATA_PATH', '.'))

train_file = str(input_data_path / "data_train.npz")
test_file = str(input_data_path / "data_test.npz")
prediction_file = str(output_data_path / "data_test_prediction.npz")


if not (os.path.isfile(train_file) and
        os.path.isfile(test_file)):
    if not os.path.isfile("input_public_data.zip"):
        !wget https://codalab.coresearch.club/my/datasets/download/37304c34-1d4a-4f43-bcb2-1fdeb37c5cba -O input_public_data.zip
    !unzip -n input_public_data.zip

In [ ]:
import numpy as np

In [ ]:
data_real = np.load(train_file, allow_pickle=True)

# This is the calorimeter response:
energy = data_real['EnergyDeposit']

# These are the quantities we want to predict
momentum = data_real['ParticleMomentum'][:,:2]
coordinate = data_real['ParticlePoint'][:,:2]

In [ ]:
print('energy.shape:', energy.shape)
print('momentum.shape:', momentum.shape)
print('coordinate.shape:', coordinate.shape)

So, we have images of 30x30 pixels and we want to predict 4 numbers for each of them: x, y, px and py.

Let's have a look at some of the images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(7, 7))
plt.subplot(221)
plt.imshow(energy[5])
plt.subplot(222)
plt.imshow(energy[50])
plt.subplot(223)
plt.imshow(energy[500])
plt.subplot(224)
plt.imshow(energy[5000]);

It's also worth knowing how the targets are distributed:

In [ ]:
plt.scatter(*momentum.T, s=5);

In [ ]:
plt.scatter(*coordinate.T, s=5);

Naive approach: can we predict the coordinates from the center of mass position of the calorimeter response?

In [ ]:
energy_density = energy / energy.sum(axis=(1, 2), keepdims=True)

cell_coords = np.stack([*np.meshgrid(
    np.arange(energy.shape[1]),
    np.arange(energy.shape[2])
)], axis=-1)[None,...]

center_of_mass = (energy_density[...,None] * cell_coords).sum(axis=(1, 2))

plt.figure(figsize=(8, 3))
plt.subplot(121)
plt.scatter(coordinate[:,0], center_of_mass[:,0], s=5)
plt.subplot(122)
plt.scatter(coordinate[:,1], center_of_mass[:,1], s=5);

Looks like the correlation isn't too strong. Maybe higher moments would give us a better picture, but we'll leave such experiments to you.

# Example solution

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
X = energy[:,..., None] # adding Channels dimension
Y = np.concatenate([coordinate, momentum], axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

In [ ]:
# Disclaimer: this might not be the best architecture for the task

inputs = tf.keras.layers.Input(shape=(30, 30, 1), name="InputImages")
tns = tf.keras.layers.Conv2D(filters=3, kernel_size=7)(inputs)
tns = tf.keras.layers.ReLU()(tns)
tns = tf.keras.layers.MaxPool2D(pool_size=(4,4))(tns)
tns = tf.keras.layers.Conv2D(filters=8, kernel_size=4)(tns)
tns = tf.keras.layers.Flatten()(tns)
tns = tf.keras.layers.Dense(32)(tns)
tns = tf.keras.layers.ReLU()(tns)
outputs = tf.keras.layers.Dense(4)(tns)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

In [ ]:
def metric_relative_mse(y_true, y_pred):
    return (
        tf.reduce_mean(tf.math.pow((y_true - y_pred), 2), axis=0) / tf.reduce_mean(tf.math.pow(y_true, 2), axis=0)
    )

def metric_relative_mse_total(y_true, y_pred):
    return tf.reduce_sum(metric_relative_mse(y_true, y_pred))

In [ ]:
learning_rate = 1e-3
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt, loss="MAE", metrics=["MAE", metric_relative_mse_total])

In [ ]:
BATCH_SIZE = 128
model.fit(X_train, Y_train, epochs=100, batch_size=BATCH_SIZE, validation_data=(X_val, Y_val))

In [ ]:
prediction_val = model.predict(X_val)
coordinate_val, momentum_val = (
    prediction_val[:, :2],
    prediction_val[:, 2:],
)
def scoring_function():
    score = 0.
    
    ParticleMomentum_sol = Y_val[:, 2:]
    ParticlePoint_sol = Y_val[:, :2]
    
    ParticleMomentum_pred = momentum_val[:, :2]
    ParticlePoint_pred = coordinate_val[:, :2]
    
    score += np.sum(np.square(ParticleMomentum_sol - ParticleMomentum_pred).mean(axis=0) / np.square(ParticleMomentum_sol).mean(axis=0))
    score += np.sum(np.square(ParticlePoint_sol - ParticlePoint_pred).mean(axis=0) / np.square(ParticlePoint_sol).mean(axis=0))
    return score

print(scoring_function())